In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xlt

ANALYSIS ON IDS

In [2]:
#import data
IDS_data = pd.read_csv(r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\Season client\Season Clients Detailed_20200926-021956.csv')
MOPA = pd.read_excel(r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\MOPA data\Bugesera MOPA data.xlsx')

In [3]:
IDS_data['SiteID'] = IDS_data['DistrictName'] +'_'+ IDS_data['SiteName']
IDS_data['SectorID'] = IDS_data['DistrictName'] +'_'+ IDS_data['GovLocationParent']
MOPA['SiteID'] = MOPA['District'] +'_'+ MOPA['Sector']
MOPA['SectorID'] = MOPA['District'] +'_'+ MOPA['Sector']

In [4]:
 #Insert parameters
Listofsectors = []
for sector in MOPA['SectorID'].unique().tolist():
     Listofsectors.append(sector)

In [5]:
#Preliminary info on IDS data on subsidized inputs

SectorDf = IDS_data [IDS_data ['SectorID'].isin(Listofsectors)]
SubsInputsdf = SectorDf[['GovLocationGrandParent','GovLocationParent','SiteName','LastName','FirstName','OAFID','NationalID','2021A_DAP kg','2021A_UREA kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg']]

#Cleaning some IDS columns
SubsInputsdf['NationalID'] =SubsInputsdf['NationalID'].astype(str).replace('\.0', '', regex=True)
SubsInputsdf['OAFID'] =SubsInputsdf['OAFID'].astype(str).replace('\.0', '', regex=True)
SubsInputsdf = SubsInputsdf.rename(columns={'GovLocationGrandParent':'GovDistrict','GovLocationParent':'GovSector','2021A_DAP kg':'DAP','2021A_NPK 17 kg':'NPK','2021A_Pan 53 kg':'PAN53', '2021A_Pan 691 kg':'PAN691', '2021A_SC 403 kg':'SC403','2021A_SC 637 kg':'SC637','2021A_UREA kg':'UREA', '2021A_WH 403 kg':'WH403','2021A_WH 505 kg':'WH505', '2021A_WH 605 kg':'WH605'})

SubsInputsdf = SubsInputsdf.rename(columns={'DAP':'I_DAP','NPK':'I_NPK','PAN53':'I_PAN53', 'PAN691':'I_PAN691', 'SC403':'I_SC403','SC637':'I_SC637','UREA':'I_UREA', 'WH403':'I_WH403','WH505':'I_WH505', 'WH605':'I_WH605'})
        

ANALYSIS ON MOPA

In [6]:
#Cleaning MOPA data
ColSplit = MOPA ['Requested Qty'].str.split(" ", n = 2, expand = True)
MOPA['Requested_qty'] = ColSplit[0].astype(float)
MOPA['Paid_qty'] = MOPA['Paid Qty'].str.extract('(\d+)').astype(float)
MOPA['Product'] = MOPA['Product'].str.upper()
MOPA = MOPA.rename(columns={'National Id':'NationalID'})
MOPA['NationalID'] = MOPA['NationalID'].astype(str)
MOPA['Product'] = MOPA['Product'].replace(['NPK 17:17:17'],['NPK'])
MOPA_dataCols = MOPA[['Name','Surname','NationalID','Product','Paid_qty']]
Products = pd.DataFrame({'Product':['DAP', 'UREA', 'NPK', 'WH403','WH505','PAN53','SC403','SC637','WH605','PAN691']})
MOPA_dataCols =pd.merge(MOPA_dataCols,Products, on='Product', how='outer').fillna(0).drop([0])


MOPA_table = pd.pivot_table(MOPA_dataCols, values='Paid_qty', index=['Name','Surname','NationalID'],columns=['Product'], aggfunc=np.sum).reset_index().fillna(0)

MOPA_table = MOPA_table.rename(columns={'DAP':'M_DAP','NPK':'M_NPK','PAN53':'M_PAN53', 'PAN691':'M_PAN691', 'SC403':'M_SC403','SC637':'M_SC637','UREA':'M_UREA', 'WH403':'M_WH403','WH505':'M_WH505', 'WH605':'M_WH605'})

In [7]:
 #COMBINING MOPA AND IDS
 IDS_MOPA = pd.merge(SubsInputsdf,MOPA_table, on='NationalID', how='left' ).drop(['Name','Surname'], axis=1)
 IDS_MOPA['SiteID'] = IDS_MOPA['GovDistrict'] +'_'+ IDS_MOPA['SiteName']

 #FINDING DIFFERENCES
IDS_MOPA['#DAP'] = IDS_MOPA['I_DAP']-IDS_MOPA['M_DAP']
IDS_MOPA['#NPK'] = IDS_MOPA['I_NPK']-IDS_MOPA['M_NPK']
IDS_MOPA['#UREA'] = IDS_MOPA['I_UREA']-IDS_MOPA['M_UREA']
IDS_MOPA['#SC403'] = IDS_MOPA['I_SC403']-IDS_MOPA['M_SC403']
IDS_MOPA['#SC637'] = IDS_MOPA['I_SC637']-IDS_MOPA['M_SC637']
IDS_MOPA['#PAN53'] = IDS_MOPA['I_PAN53'] - IDS_MOPA['M_PAN53']
IDS_MOPA['#PAN691'] = IDS_MOPA['I_PAN691'] - IDS_MOPA['M_PAN691'] 
IDS_MOPA['#WH505'] = IDS_MOPA['I_WH505']-IDS_MOPA['M_WH505']
IDS_MOPA['#WH403'] = IDS_MOPA['I_WH403']-IDS_MOPA['M_WH403']
IDS_MOPA['#WH605'] = IDS_MOPA['I_WH605']-IDS_MOPA['M_WH605']

In [8]:
MeltIDS_MOPA = IDS_MOPA.melt(['SiteID','GovDistrict', 'GovSector', 'SiteName', 'LastName', 'FirstName','OAFID', 'NationalID'],['I_DAP', 'I_UREA', 'I_NPK', 'I_PAN53','I_PAN691', 'I_SC403', 'I_SC637', 'I_WH403', 'I_WH505', 'I_WH605', 'M_DAP', 'M_NPK', 'M_PAN53','M_PAN691', 'M_SC403', 'M_SC637', 'M_UREA','M_WH403', 'M_WH505', 'M_WH605', '#DAP', '#NPK','#UREA', '#SC403', '#SC637','#PAN53', '#PAN691', '#WH505','#WH403', '#WH605'],'Product','Quantity').groupby('Product')['Quantity'].sum().reset_index()


In [14]:
Site = IDS_MOPA[IDS_MOPA['SiteID']=='Bugesera_Biryogo']
ProductsTaken = MeltIDS_MOPA[MeltIDS_MOPA['Quantity']!=0]
ListOfproducts = sorted(ProductsTaken['Product'].tolist(), reverse=True)
CustomList = ['I_DAP','I_UREA','I_NPK','I_PAN53','I_PAN691', 'I_SC403', 'I_SC637', 'I_WH403', 'I_WH505', 'I_WH605','M_DAP','M_UREA','M_NPK','M_PAN53','M_PAN691', 'M_SC403', 'M_SC637', 'M_WH403', 'M_WH505', 'M_WH605','#DAP','#UREA','#NPK','#PAN53','#PAN691', '#SC403', '#SC637', '#WH403', '#WH505', '#WH605']

zipped_lists = zip(ListOfproducts, CustomList)
sorted_zipped_lists = sorted(zipped_lists,reverse=True)
sorted_list1 = [element for _, element in sorted_zipped_lists]
sorted_list1
OtherColumns = Site.columns.tolist()[:7]
AllColumns = OtherColumns + sorted_list1
SiteDf = Site[AllColumns]
District = SiteDf['GovDistrict'].unique().tolist()[0]
Sector = SiteDf['GovSector'].unique().tolist()[0]
Site = SiteDf['SiteName'].unique().tolist()[0]
FinalSiteDf = SiteDf.drop(['GovDistrict', 'GovSector', 'SiteName'], axis=1)
FinalSiteDf = FinalSiteDf.fillna('-')

In [21]:
diff = set(CustomList) - set(ListOfproducts)
listDiff = list(diff)
CustomList.remove(listDiff)

CustomList

ValueError: list.remove(x): x not in list

In [117]:
save_name= 'gasana001' +'.xlsx'
writer = pd.ExcelWriter(save_name, engine='xlsxwriter')

import pandas.io.formats.excel
pandas.io.formats.excel.header_style = None
FinalSiteDf.to_excel(writer, header=True, sheet_name='site', startrow=8, index=False)



workbook = writer.book
workbook.formats[0].set_font_size(10)
worksheet = writer.sheets['site']

title_format = workbook.add_format({'bold': True,
                                    'text_wrap':True, 
                                    'border':5,
                                    'font_size':18}) 
worksheet.set_column(4, 18, 4)
worksheet.set_column(0, 1, 14.5)
worksheet.set_column(2, 2, 10.5)
#worksheet.set_column(1, 2, 20)
worksheet.set_column(3, 3, 15.5)

worksheet.set_row(9, 24)

title_format.set_align('center')
title_format.set_align('vcenter')
worksheet.merge_range('A2:H2','IHUZAMAKURU RYA IDS NA TMS RYA 2019 A\n KUMENYESHA INYONGERAMUSARURO ZABUZE',title_format)

id_format = workbook.add_format({'bold': True,
                                    'text_wrap':True, 
                                    'font_size':12})
id_format.set_align('top')
id_format.set_align('left')

head_format = workbook.add_format({'bold': True,
                                    'text_wrap':True, 
                                    'font_size':10})
head_format.set_rotation(90)
for col_num, value in enumerate(FinalSiteDf.columns.values[4:]):
    worksheet.write(8, col_num+4, value, head_format)




worksheet.set_row(8, 50, head_format)

#identity titles and their cell locations
worksheet.merge_range('A4:B4', 'Akarere',id_format)
worksheet.merge_range('A5:B5', 'Umurenge',id_format)
worksheet.merge_range('A6:B6', "Site",id_format)


#identity values and their cell locations
worksheet.merge_range('C4:D4',District, id_format)
worksheet.merge_range('C5:D5',Sector, id_format)
worksheet.merge_range('C6:D6',Site, id_format)

#add borders to the table
border_format = workbook.add_format({'border':1})
bor_last_row = str(len(FinalSiteDf['NationalID'])+8)
bor_range = 'A9' + ':' + 'S'+ bor_last_row
worksheet.conditional_format(bor_range, {'type':'no_blanks',
                                        'format':border_format})

writer.save()






In [111]:
FinalSiteDf.columns.values[4:]

array(['M_WH403', 'M_UREA', 'M_SC403', 'M_NPK', 'M_DAP', 'I_WH403',
       'I_UREA', 'I_SC403', 'I_NPK', 'I_DAP', '#WH403', '#UREA', '#SC403',
       '#NPK', '#DAP'], dtype=object)